引入依賴套件
----------

In [79]:
import json
import collections

import jieba

import numpy as np
import pandas as pd

In [50]:
data_df = pd.read_csv('./data.csv', encoding = 'utf-8')
data_df.head() #取得前五名資料並輸出

,摘要,標註
0,中國福建省武平縣13日因連日強降雨，導致該縣平川街道東門市場旁的山坡發生山崩，多輛停在路旁的...,土石流
1,中國南方多省自6月上旬以來持續暴雨，各地紛紛出現洪水，統計至今已釀61死、14人失蹤、531...,水災
2,日本南部鹿兒島屋久島鎮，昨天一個下午降雨420毫米雨量，打破當地50年來最大單日降雨量，由於...,大雨
3,日本沖繩縣與那國島距離宜蘭僅有111公里，今日上午遭遇了一場暴雨，由於降雨量極大，日本氣象廳...,大雨
4,日本沖繩縣與那國島距離宜蘭僅有111公里，今日上午遭遇了一場暴雨，由於降雨量極大，日本氣象廳...,大雨


In [51]:
def segmentation(sentence: str) -> list:
    return jieba.lcut(sentence)

將摘要進行中文斷詞
---------------------

In [52]:
data_df['segmentation'] = data_df['摘要'].apply(segmentation)
data_df.head()

,摘要,標註,segmentation
0,中國福建省武平縣13日因連日強降雨，導致該縣平川街道東門市場旁的山坡發生山崩，多輛停在路旁的...,土石流,"[中國, 福建省, 武平, 縣, 13, 日因, 連日強, 降雨, ，, 導致, 該, 縣,..."
1,中國南方多省自6月上旬以來持續暴雨，各地紛紛出現洪水，統計至今已釀61死、14人失蹤、531...,水災,"[中國, 南方, 多省, 自, 6, 月, 上旬, 以來, 持續, 暴雨, ，, 各地, 紛..."
2,日本南部鹿兒島屋久島鎮，昨天一個下午降雨420毫米雨量，打破當地50年來最大單日降雨量，由於...,大雨,"[日本, 南部, 鹿兒島, 屋久, 島鎮, ，, 昨天, 一個, 下午, 降雨, 420, ..."
3,日本沖繩縣與那國島距離宜蘭僅有111公里，今日上午遭遇了一場暴雨，由於降雨量極大，日本氣象廳...,大雨,"[日本, 沖, 繩縣, 與, 那國島, 距離, 宜蘭僅, 有, 111, 公里, ，, 今日..."
4,日本沖繩縣與那國島距離宜蘭僅有111公里，今日上午遭遇了一場暴雨，由於降雨量極大，日本氣象廳...,大雨,"[日本, 沖, 繩縣, 與, 那國島, 距離, 宜蘭僅, 有, 111, 公里, ，, 今日..."


將斷詞結果整合成陣列，並計算最大長度的詞句
-----------------------------------

In [97]:
import sys

def get_all_vocab_from_data(data, colunm_name):
    train_vocab_list = []
    min_cut_query_length = sys.maxsize
    max_cut_query_length = 0
    for cut_query in data[colunm_name]:
        query_length = len(cut_query)
        max_cut_query_length = max(max_cut_query_length, query_length)
        min_cut_query_length = min(min_cut_query_length, query_length)
        train_vocab_list += cut_query
    return train_vocab_list, max_cut_query_length, min_cut_query_length

In [98]:
train_vocab_list, max_cut_query_length, min_cut_query_length = get_all_vocab_from_data(data_df, 'segmentation')

In [99]:
print('最長斷詞長度: {}'.format(max_cut_query_length))
print('最短斷詞長度: {}'.format(min_cut_query_length))
print('斷詞結果: {} 詞 (重複)'.format(len(train_vocab_list)))
print('斷詞結果: {} 詞 (不重複)'.format(len(set(train_vocab_list))))

最長斷詞長度: 178
最短斷詞長度: 16
斷詞結果: 4652 詞 (重複)
斷詞結果: 1178 詞 (不重複)


In [83]:
train_vocab_counter = collections.Counter(train_vocab_list)
train_vocab_counter.most_common(5)

[('，', 395), ('。', 118), ('（', 100), ('）', 100), ('地震', 86)]

In [67]:
label_counter = collections.Counter(data_df['標註'])
label_counter.most_common() #最常用的前五名

[('地震', 42), ('土石流', 4), ('大雨', 3), ('水災', 2)]

統計沒有用途的詞（低使用率）
-------------

In [76]:
def statistic_zero_usage_word(counter: collections.Counter) -> int:
    counter = 0
    for word, times in train_vocab_counter.items():
        if times <=1:
            counter+=1
    return counter
train_data_times_zero = statistic_zero_usage_word(train_vocab_counter)
print('只使用到一次的詞: {}種'.format(train_data_times_zero))
print('佔全部 {} %'.format(train_data_times_zero/len(train_vocab_counter)*100))

只使用到一次的詞: 646種
佔全部 54.83870967741935 %


讀取和寫入JSON file的函數
-----------

In [100]:
def read_from_json(path: str) -> dict:
    with open(path, encoding='utf-8') as f:
        return json.load(f)
    
def write_to_json(path: str, data: dict):
    with open(path, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False)

轉換Counter成字典並儲存成data_count.json
--------

In [101]:
write_to_json('./data_count.json', dict(train_vocab_counter))

補充：讀取data_count.json後再轉換回Counter
------

In [104]:
data = read_from_json('./data_count.json')
counter = collections.Counter(data)
counter.most_common(5)

[('，', 395), ('。', 118), ('（', 100), ('）', 100), ('地震', 86)]

建立分詞索引表並儲存到word_index.json
--------

In [117]:
def counter_to_key_index_table(counter: collections.Counter) -> dict:
    index = 0
    result = {}
    for key, value in counter.most_common(): #這樣的做法能讓最後建出的表依照使用順序排序(越常使用越前面)
        result[key] = index
        index+=1
    return result
word_to_index = counter_to_key_index_table(train_vocab_counter)
write_to_json('word_index.json', word_to_index) #儲存到word_index.json
print('分詞索引表長度: {}'.format(len(word_to_index)))

分詞索引表長度: 1178


補充：將word_index.json讀入後翻轉成index: word的形式
----------

In [121]:
word_to_index = read_from_json('./word_index.json')
index_to_word = {index: word for word, index in word_to_index.items()}
index_to_word

#ref: https://stackoverflow.com/questions/483666/python-reverse-invert-a-mapping

{0: '，',
 1: '。',
 2: '（',
 3: '）',
 4: '地震',
 5: '的',
 6: '規模',
 7: '發生',
 8: '公里',
 9: ' ',
 10: '強震',
 11: '、',
 12: '在',
 13: '時間',
 14: '深度',
 15: '於',
 16: '芮氏',
 17: '地質',
 18: '震央',
 19: '美國',
 20: 'USGS',
 21: '震源',
 22: '目前',
 23: '位',
 24: '日',
 25: '台灣',
 26: '當地',
 27: '下午',
 28: '時',
 29: '也',
 30: '調查',
 31: '有',
 32: '點',
 33: '人',
 34: '度',
 35: '傳出',
 36: '所',
 37: '印尼',
 38: '今',
 39: '根據',
 40: '6.1',
 41: '報導',
 42: '指出',
 43: '日本',
 44: '地區',
 45: '調查局',
 46: '分',
 47: '警報',
 48: '造成',
 49: '今天',
 50: '2',
 51: '僅',
 52: '10',
 53: '外海',
 54: '海嘯',
 55: '處',
 56: '與',
 57: '表示',
 58: '傷亡',
 59: '死亡',
 60: '今天上午',
 61: '觀測',
 62: '海域',
 63: '消息',
 64: '最大',
 65: '3',
 66: '氣象',
 67: '5.4',
 68: '近海',
 69: '為',
 70: '發布',
 71: '倒塌',
 72: '廳',
 73: '未',
 74: '淺層',
 75: '4',
 76: '5',
 77: '到',
 78: '11',
 79: '極淺層',
 80: '山崩',
 81: '引發',
 82: '或',
 83: '約',
 84: '《',
 85: '》',
 86: '瀑布',
 87: '震度',
 88: '1',
 89: '數據',
 90: '年',
 91: '機構',
 92: '目島',
 93: '40',
 94: 